In [135]:
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import cess_esp
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from collections import Counter

nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('cess_esp')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')


[nltk_data] Downloading package words to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package cess_esp to
[nltk_data]     /Users/fredericksalazar/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fredericksalazar/nltk_d

True

# Limpieza General de datos

se realizará un limpieza general de los archivos a procesar con base en las siguientes reglas definidas:

- todo estará en minusculas
- se reemplazará caracter - por vacio
- se eliman los saltos de linea \n por vacio


In [121]:
pathArchivo = './archivos/petro.pdf.txt'

archivo = open(pathArchivo, 'r', encoding='utf8')

lineaMod = ''
flag = False
text = ''

while(True):
    linea = archivo.readline()
    text = text+linea
    if not linea:
        break
    
archivo.close

<function TextIOWrapper.close()>

In [122]:
#Reemplazamos caracteres incomodos que son usados en el texto, convertimos todo a minusculas para mejor procesamiento

text = text.replace('-', '\n')
text = text.replace('\n', '')

text = text.lower()


# Tokenización

Como paso fundamental en el NLP la tokenización nos permite tomar un texto entero y dividirlo en cada una de las palabras que lo componen, esta división permitirá posteriormente entender las palabras usadas.

In [124]:
#hacemos uso del modulo word_tokenize de nltk para separar cada una de las palabras usadas

texto_tokenizado = word_tokenize(text)
texto_tokenizado

['colombia',
 'potencia',
 'mundial',
 'de',
 'la',
 'vidallegó',
 'la',
 'hora',
 'de',
 'cambiar',
 '.',
 'colombia',
 'será',
 'potencia',
 'mundial',
 'de',
 'la',
 'vida',
 'mediante',
 'un',
 'pacto',
 'histórico',
 'que',
 'de',
 'manera',
 'urgente',
 'concrete',
 'un',
 'nuevo',
 'contrato',
 'social',
 'para',
 'el',
 'buen',
 'vivir',
 'y',
 'el',
 'vivir',
 'sabroso',
 'con',
 'todas',
 'las',
 'diversidades',
 'de',
 'la',
 'nación',
 'para',
 'entrar',
 'por',
 'fin',
 'en',
 'una',
 'era',
 'de',
 'paz',
 'cuyo',
 'rostro',
 'sea',
 'una',
 'democracia',
 'multicolor',
 ',',
 'expresión',
 'de',
 'los',
 'mandatos',
 'ciudadanos',
 ',',
 'que',
 'haga',
 'realidad',
 'la',
 'igualdad',
 ',',
 'una',
 'economía',
 'productiva',
 'que',
 'genere',
 'riqueza',
 'basada',
 'en',
 'el',
 'trabajo',
 'y',
 'la',
 'protección',
 'de',
 'la',
 'casa',
 'común',
 '.',
 'este',
 'programa',
 'para',
 'el',
 'cambio',
 ',',
 'construido',
 'desde',
 'los',
 'territorios',
 'y',
 'l

In [125]:
print('Total Aproximado de palabras usadas en el texto: ',len(texto_tokenizado))

Total Aproximado de palabras usadas en el texto:  28807


# Eliminando Stop Words

Vamos a quitar las stop words que no aportan valor al texto 

In [126]:
es_stop_words = stopwords.words('spanish')

print(len(es_stop_words))

313


In [127]:
#Creamos una lista sin las palabras stop word del español

texto_sin_stp = [words for words in texto_tokenizado if words not in es_stop_words]

print('total palabras con stop words : ', len(texto_sin_stp))

total palabras con stop words :  16393


In [128]:
#vamos a crear una lista de solo palabras usadas en el documento
ps = PorterStemmer()

lista_palabras = [palabra for palabra in texto_sin_stp if palabra.isalnum()]

print('total palabras : ', len(lista_palabras))

total palabras :  13822


# Contador de palabras

Procedemos a contar cada una de las palabras que existen dentro de la lista de palabras final.

In [129]:
contador = Counter(lista_palabras)

In [130]:
contador.most_common(30)

[('sistema', 103),
 ('vida', 92),
 ('economía', 85),
 ('nacional', 85),
 ('salud', 73),
 ('mujeres', 71),
 ('derechos', 70),
 ('social', 66),
 ('trabajo', 62),
 ('colombia', 61),
 ('protección', 61),
 ('política', 59),
 ('paz', 56),
 ('país', 54),
 ('acceso', 53),
 ('hacia', 48),
 ('derecho', 48),
 ('así', 46),
 ('cambio', 45),
 ('condiciones', 44),
 ('cuidado', 44),
 ('educación', 44),
 ('sociedad', 42),
 ('productiva', 41),
 ('desarrollo', 39),
 ('participación', 38),
 ('cultura', 38),
 ('gobierno', 38),
 ('construcción', 37),
 ('personas', 37)]

# etiquetado de palabras

en el caso del etiquetado hemos usado el postagger por defecto encontrando problemas dado que soporta mejor el ingles, en este caso vamos a usar un nuevo etiquetador que es Stanford que si soporta el idioma el español.

vamos a aplicar una técnica de etiquetado al texto de plan de gobierno a fin de comprender el uso de las diferentes palabras y que tipo son:

- JJ -> Adjetivos
- NN -> Sustantivos
- RB -> Adverbios
- PRP -> Pronombres
- VB -> Verbos

In [136]:
taged_esp = cess_esp.tagged_sents()

taged_esp

[[('El', 'da0ms0'), ('grupo', 'ncms000'), ('estatal', 'aq0cs0'), ('Electricité_de_France', 'np00000'), ('-Fpa-', 'Fpa'), ('EDF', 'np00000'), ('-Fpt-', 'Fpt'), ('anunció', 'vmis3s0'), ('hoy', 'rg'), (',', 'Fc'), ('jueves', 'W'), (',', 'Fc'), ('la', 'da0fs0'), ('compra', 'ncfs000'), ('del', 'spcms'), ('51_por_ciento', 'Zp'), ('de', 'sps00'), ('la', 'da0fs0'), ('empresa', 'ncfs000'), ('mexicana', 'aq0fs0'), ('Electricidad_Águila_de_Altamira', 'np00000'), ('-Fpa-', 'Fpa'), ('EAA', 'np00000'), ('-Fpt-', 'Fpt'), (',', 'Fc'), ('creada', 'aq0fsp'), ('por', 'sps00'), ('el', 'da0ms0'), ('japonés', 'aq0ms0'), ('Mitsubishi_Corporation', 'np00000'), ('para', 'sps00'), ('poner_en_marcha', 'vmn0000'), ('una', 'di0fs0'), ('central', 'ncfs000'), ('de', 'sps00'), ('gas', 'ncms000'), ('de', 'sps00'), ('495', 'Z'), ('megavatios', 'ncmp000'), ('.', 'Fp')], [('Una', 'di0fs0'), ('portavoz', 'nccs000'), ('de', 'sps00'), ('EDF', 'np00000'), ('explicó', 'vmis3s0'), ('a', 'sps00'), ('EFE', 'np00000'), ('que', 'c

In [131]:
lista_palabras_tag = nltk.pos_tag(lista_palabras)

lista_palabras_tag

[('colombia', 'NN'),
 ('potencia', 'VBZ'),
 ('mundial', 'JJ'),
 ('vidallegó', 'NN'),
 ('hora', 'NN'),
 ('cambiar', 'NN'),
 ('colombia', 'NN'),
 ('potencia', 'VBP'),
 ('mundial', 'JJ'),
 ('vida', 'NN'),
 ('mediante', 'NN'),
 ('pacto', 'NN'),
 ('histórico', 'NN'),
 ('manera', 'NN'),
 ('urgente', 'JJ'),
 ('concrete', 'JJ'),
 ('nuevo', 'NN'),
 ('contrato', 'NN'),
 ('social', 'JJ'),
 ('buen', 'NN'),
 ('vivir', 'NN'),
 ('vivir', 'NN'),
 ('sabroso', 'NN'),
 ('todas', 'NN'),
 ('diversidades', 'NNS'),
 ('nación', 'VBP'),
 ('entrar', 'JJ'),
 ('fin', 'NN'),
 ('paz', 'NN'),
 ('cuyo', 'NN'),
 ('rostro', 'NN'),
 ('democracia', 'NN'),
 ('multicolor', 'NN'),
 ('expresión', 'NN'),
 ('mandatos', 'NN'),
 ('ciudadanos', 'NN'),
 ('haga', 'NN'),
 ('realidad', 'NN'),
 ('igualdad', 'NN'),
 ('economía', 'NN'),
 ('productiva', 'NN'),
 ('genere', 'RB'),
 ('riqueza', 'VBZ'),
 ('basada', 'JJ'),
 ('trabajo', 'NN'),
 ('protección', 'NN'),
 ('casa', 'NN'),
 ('común', 'NN'),
 ('programa', 'NN'),
 ('cambio', 'NN'),
 ('

In [112]:
ne_palabras = nltk.ne_chunk(lista_palabras_tag)

print(ne_palabras)

(S
  colombia/JJ
  potenciamundial/JJ
  vidallegó/NN
  hora/NN
  cambiar/NN
  colombia/NN
  potencia/VBP
  mundial/JJ
  vida/NN
  mediante/NN
  pacto/NN
  histórico/NN
  manera/NN
  urgente/JJ
  concrete/JJ
  nuevo/NN
  contrato/NN
  social/JJ
  buen/NN
  vivir/NN
  vivir/NN
  sabroso/NN
  todas/NN
  diversidades/NNS
  nación/VBP
  entrar/JJ
  fin/NN
  paz/NN
  cuyo/NN
  rostro/NN
  democracia/NN
  multicolor/NN
  expresión/NN
  mandatos/NN
  ciudadanos/NN
  haga/NN
  realidad/NN
  igualdad/NN
  economía/NN
  productiva/NN
  genere/RB
  riqueza/VBZ
  basada/JJ
  trabajo/NN
  protección/NN
  casa/NN
  común/NN
  programa/NN
  cambio/NN
  construido/NN
  territorios/NNS
  voces/NNS
  gentes/NNS
  concebido/VBP
  inicio/JJ
  transición/NN
  inmediato/NN
  hará/NN
  posible/JJ
  vida/NN
  digna/NN
  superación/NN
  violencia/NN
  justicia/NN
  social/JJ
  climática/NN
  tiempo/JJ
  consolidan/NN
  condiciones/NNS
  permanentes/VBZ
  paz/JJ
  grande/NN
  permita/NN
  toda/NN
  sociedad/NN
 

# Generación de texto depurado

In [85]:

texto_depurado = ''

for palabra in texto_tokenizado:
    if palabra.isalpha():
        texto_depurado = texto_depurado+' '+palabra
    


In [86]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [90]:
ps.stem('ordenaremos')

'ordenaremo'

In [10]:
fileText = open('./archivos/clean_petro.txt', 'w')
fileText.write(text)
fileText.close()